# Car Tracker 


This notebook shows how to build a simple car dwell time tracker a pretrained SSD Mobilenet Tensorflow Object Detection Model and a Centroid Tracker.

By completing this notebook, you will learn:
* How to write a Python script for your app that takes in camera streams, performs inference, and outputs results
* How to use a Tensorflow Object Detecrtion model with Panorama
* How to bundle additional Python files and libraries with your container
* How to build a simple car tracker
* How to test your code using the Test Utility, which saves you build and deploy time
* How to programmatically package and deploy applications using the Panorama CLI

--- 

1. [Prerequisites](#Prerequisites)
1. [Set up](#Set-up)
1. [Import model](#Import-model)
1. [Write and test app code](#Write-and-test-app-code)
1. [Package app](#Package-app)
1. [Deploy app to device](#Deploy-app-to-device)

# Prerequisites

1. In a terminal session on this Jupyter notebook server, run `aws configure`. This allows this notebook server to access Panorama resources on your behalf.

# Set Up
Import libraries for use with this notebook environment, you do not need these libraries when you write your application code. Run these 3 cells every time you update your app code and restart your kernel.

In [ ]:
import sys
import os
import time
import json

import boto3
import sagemaker

import matplotlib.pyplot as plt
from IPython.core.magic import register_cell_magic

sys.path.insert( 0, os.path.abspath( "../common/test_utility" ) )
import panorama_test_utility

# instantiate boto3 clients
s3_client = boto3.client('s3')
panorama_client = boto3.client('panorama')

# configure matplotlib
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,20)

# register custom magic command
@register_cell_magic
def save_cell(line, cell):
    'Save python code block to a file'
    with open(line, 'wt') as fd:
        fd.write(cell)

## Notebook parameters

Global constants that help the notebook create Panorama resources on your behalf.

In [ ]:
# Device ID, should look like: device-oc66nax4cgzwhyuaeyifrqowue
DEVICE_ID = input( 'DEVICE_ID (format: device-*)' ).strip()

# Enter your S3 bucket info here
S3_BUCKET = input( 'S3_BUCKET' ).strip()

# Enter your desired AWS region
AWS_REGION = input( 'AWS_REGION (e.g. us-east-1)' ).strip()  

ML_MODEL_FNAME = 'ssd_mobilenet_v2_coco'

In [ ]:
# application name
app_name = 'car_tracker_app'

## package names and node names
code_package_name = 'CAR_TRACKER_CODE'
model_package_name = 'CAR_SSD_TF_MODEL'
camera_node_name = 'RTSP_STREAM'

# model node name, raw model path (without platform dependent suffics), and input data shape
model_node_name = "model_node"
model_file_basename = "./models/" + ML_MODEL_FNAME
model_data_shape = '{"image_tensor":[1,300,300,3]}'

# video filename to simulate camera stream
videoname = '../common/test_utility/videos/Car-in-the-mist_coverr.mp4'

# AWS account ID
account_id = boto3.client("sts").get_caller_identity()["Account"]


## Set up application

Every application uses the creator's AWS Account ID as the prefix to uniquely identifies the application resources. Running `panorama-cli import-application` replaces the generic account Id with your account Id.

In [ ]:
!cd ./car_tracker_app && panorama-cli import-application

# Import model

We need to compile and import the model twice. Once for testing with this notebook server and once for deploying to the Panorama device.

While working with the Panorama sample code, we provide pretrained models for you to use. Locally, models are stored in `./models`. This step downloads the model artifacts from our Amazon S3 bucket to the local folder. If you want to use your own models, put your tar.gz file into the `./models` folder.

### Prepare model for testing with notebook server

In [ ]:
# Downloads pretrained model for this sample.
# This step takes some time, depending on your network environment.
panorama_test_utility.download_sample_model( ML_MODEL_FNAME, "./models" )

In [ ]:
# Compile the model to run with test-utility.
# This step takes 30 mins ~ 40 mins.
%run ../common/test_utility/panorama_test_utility_compile.py \
\
--s3-model-location s3://{S3_BUCKET}/{app_name}/ \
\
--model-node-name model_node \
--model-file-basename ./models/{ML_MODEL_FNAME} \
--model-data-shape '{model_data_shape}' \
--model-framework TENSORFLOW

### Prepare model for deploying to Panorama device


In [ ]:
model_asset_name = 'model_asset'
model_package_path = f'packages/{account_id}-{model_package_name}-1.0'
model_descriptor_path = f'packages/{account_id}-{model_package_name}-1.0/descriptor.json'

In [ ]:
!cd ./car_tracker_app && panorama-cli add-raw-model \
    --model-asset-name {model_asset_name} \
    --model-s3-uri s3://{S3_BUCKET}/{app_name}/{model_node_name}/{ML_MODEL_FNAME}.tar.gz \
    --descriptor-path {model_descriptor_path}  \
    --packages-path {model_package_path}

# Write and test app code

Every app has an entry point script, written in Python that pulls the frames from camera streams, performs inference, and send the results to the desired location. This file can be found in `your_app/packages/code_node/src/app.py`. Below, you will iterate on the code from within the notebook environment. The entry point file will be updated everytime you run the next notebook cell thanks to the `%%save_cell`. This is a  magic command to update the contents of the entry point script. 

After updating the entry point script, use the Test Utility Run command (panorama_test_utility_run.py) command to simulate the application.

### Iterating on Code Changes

To iterate on the code:
1. Interrupt the kernel if application is still running.
2. Make changes in the next cell, and run the cell to update the entry point script. 
3. Run the panorama_test_utility_run.py again.

**CHANGE VIDEO** : For you to change video, please set the file path to the --video-file argument of the panorama_test_utility_run.py command.

In [ ]:
%%save_cell ./{app_name}/packages/{account_id}-{code_package_name}-1.0/src/app.py

import json
import logging
import time
from logging.handlers import RotatingFileHandler

import boto3
import cv2
import numpy as np
import panoramasdk
import datetime
from CentroidTracker import CentroidTracker

class Application(panoramasdk.node):
    def __init__(self):
        """Initializes the application's attributes with parameters from the interface, and default values."""

        self.MODEL_NODE = "model_node"
        self.MODEL_DIM = 300
        self.frame_num = 0
        self.tracker = CentroidTracker(maxDisappeared=80, maxDistance=90)
        self.tracked_objects = []
        self.tracked_objects_start_time = dict()
        self.tracked_objects_duration = dict()

        try:
            # Parameters
            logger.info('Configuring parameters.')
            self.threshold = self.inputs.threshold.get()
            
            # Desired class
            self.classids = [3.]

        except:
            logger.exception('Error during initialization.')
        finally:
            logger.info('Initialiation complete.')

    def process_streams(self):
        """Processes one frame of video from one or more video streams."""
        self.frame_num += 1
        logger.debug(self.frame_num)

        # Loop through attached video streams
        streams = self.inputs.video_in.get()
        for stream in streams:
            self.process_media(stream)

        self.outputs.video_out.put(streams)

    def process_media(self, stream):
        """Runs inference on a frame of video."""
        image_data = preprocess(stream.image, self.MODEL_DIM)
        logger.debug(image_data.shape)

        # Run inference
        inference_results = self.call({"image_tensor":image_data}, self.MODEL_NODE)

        # Process results (object deteciton)
        self.process_results(inference_results, stream)

    def process_results(self, inference_results, stream):
        """Processes output tensors from a computer vision model and annotates a video frame."""
        if inference_results is None:
            logger.warning("Inference results are None.")
            return
        
        w,h,c = stream.image.shape

        conf_scores = None
        classes = None
        bboxes = None
        rects = []

        for det in inference_results:
            if det.shape[-1] == 4:
                bboxes = det[0]
            elif det.shape[-1] == 100:
                if det[0][0] >= 1:
                    classes = det[0]
                else:
                    conf_scores = det[0]
        
        for a in range(len(conf_scores)):
            if conf_scores[a] * 100 > self.threshold and classes[a] in self.classids:
                (top, left, bottom, right) = bboxes[a]
                rects.append([left*w, top*h, right*w, bottom*h])
                stream.add_rect(left, top, right, bottom)
                
        rects = np.array(rects)
        rects = rects.astype(int)
        objects = self.tracker.update(rects)
        
        logger.info('Tracking {} cars'.format(len(objects)))
        
        for (objectID, bbox) in objects.items():
            x1, y1, x2, y2 = bbox
            x1 = int(x1)
            y1 = int(y1)
            x2 = int(x2)
            y2 = int(y2)

            if objectID not in self.tracked_objects:
                self.tracked_objects.append(objectID)
                self.tracked_objects_start_time[objectID] = datetime.datetime.now()
                self.tracked_objects_duration[objectID] = 0
            else:
                time_diff = datetime.datetime.now() - self.tracked_objects_start_time[objectID]
                sec = time_diff.total_seconds()
                self.tracked_objects_duration[objectID] = sec
            
            duration = self.tracked_objects_duration[objectID]
            
            logger.info('CarId: {} at ({},{}) for {}'.format(objectID, x1, y1, duration))
            stream.add_rect(x1/w, y1/h, x2/w, y2/h)
            stream.add_label('{}s'.format(str(duration)), x1/w, y1/h)

def preprocess(img, size):
    """Resizes and normalizes a frame of video."""
    resized = cv2.resize(img, (size, size))
    x1 = np.asarray(resized)
    x1 = np.expand_dims(x1, 0)
    return x1

def get_logger(name=__name__,level=logging.INFO):
    logger = logging.getLogger(name)
    logger.setLevel(level)
    handler = RotatingFileHandler("/opt/aws/panorama/logs/app.log", maxBytes=100000000, backupCount=2)
    formatter = logging.Formatter(fmt='%(asctime)s %(levelname)-8s %(message)s',
                                    datefmt='%Y-%m-%d %H:%M:%S')
    handler.setFormatter(formatter)
    logger.addHandler(handler)
    return logger

def main():
    try:
        logger.info("INITIALIZING APPLICATION")
        app = Application()
        logger.info("PROCESSING STREAMS")
        while True:
            app.process_streams()
    except:
        logger.exception('Exception during processing loop.')

logger = get_logger(level=logging.INFO)
main()

In [ ]:
# Run the application with test-utility.
#
# As '--output-pyplot' option is specified, this command simulates HDMI output with pyplot rendering in the output cell.
# In order to see console output (stdout/stderr) from the application, please remove the --output-pyplot option.
#
%run ../common/test_utility/panorama_test_utility_run.py \
\
--app-name {app_name} \
--code-package-name {code_package_name} \
--model-package-name {model_package_name} \
--camera-node-name {camera_node_name} \
--model-node-name {model_node_name} \
--model-file-basename {model_file_basename} \
--video-file {videoname} \
--py-file ./{app_name}/packages/{account_id}-{code_package_name}-1.0/src/app.py \
--output-pyplot

# Package app

Updates the app to be deployed with the recent code

In [ ]:
py_file_name = 'app.py'
panorama_test_utility.update_package_descriptor( app_name, account_id, code_package_name, py_file_name )

### Build app with container

In [ ]:
container_asset_name = 'code_asset'

In [ ]:
%%capture captured_output

# Building container image.This process takes time (5min ~ 10min)
# FIXME : without %%capture, browser tab crashes because of too much output from the command.

!cd ./car_tracker_app && panorama-cli build \
    --container-asset-name {container_asset_name} \
    --package-path packages/{account_id}-{code_package_name}-1.0

In [ ]:
stdout_lines = captured_output.stdout.splitlines()
stderr_lines = captured_output.stderr.splitlines()
print("     :")
print("     :")
for line in stdout_lines[-30:] + stderr_lines[-30:]:
    print(line)

### Upload application to Panorama for deploying to devices

### Update camera streams

In the AWS Panorama console, you can select the camera streams, but programmatically, you need to define the camera stream info for the cameras you are using with the app.

Open the ```package.json``` in ```packages/<account_number>-RTSP_STREAM-1.0``` and update the camera username, password and URL. After you have updated your camera credentials, run package-application. You can override this camera stream when you deploy the app.

In [ ]:
# This step takes some time, depending on your network environment.
!cd ./car_tracker_app && panorama-cli package-application

### Ready for deploying to a device

Congrats! Your app is now ready to deploy to a device. Next, you can continue in this notebook to deploy the app programmatically or you can go to the Panorama console and deploying using a GUI. The GUI makes it easier to select camera streams and select the devices you want to deploy to. Programmatic deployment is faster to complete and easier to automate.

# Deploy app to device

Let's make sure the device we are deploying to is available.

In [ ]:
response = panorama_client.describe_device(
    DeviceId= DEVICE_ID
)

print('You are deploying to Device: {}'.format(response['Name']))

#### Deploy Application

In [ ]:
response = panorama_test_utility.deploy_app(DEVICE_ID, app_name, role=None)

application_instance_id = response['ApplicationInstanceId']

response

# Clean up

In [ ]:
panorama_test_utility.remove_application( DEVICE_ID, application_instance_id )